## Running AI Red Teaming Agent ##
- This item is in preview so with that being said this code and syntax can likely change once this goes into "GA"
- Region Support (East US 2, Sweden Central, France Central, Switzerland West)
- Ensure you authenticate via CLI when launching this (az login --use-device-code)
- For the evaluation upload if you run into issues ensure Credential (assigned to user is Storage Blob Data Contributor to associated storage account or check network settings if segmented can be a challenge)

In [ ]:
%pip install azure-ai-evaluation[redteam]
%pip install python-dotenv
%pip install azure-ai-projects

In [4]:
# Azure imports
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.evaluation.red_team import RedTeam, RiskCategory


# Load environment variables from .env file
load_dotenv()

# Azure AI Project Information
azure_ai_project = {
    "subscription_id": os.environ.get("AZURE_SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("AZURE_RESOURCE_GROUP_NAME"),
    "project_name": os.environ.get("AZURE_PROJECT_NAME"),
}

# Instantiate your AI Red Teaming Agent
red_team_agent = RedTeam(
    azure_ai_project=azure_ai_project, # required
    credential=DefaultAzureCredential(), # required
    risk_categories=[ # optional, defaults to all four risk categories
        RiskCategory.Violence,
    ], 
    num_objectives=5, # optional, defaults to 10
)

## Creating a target ##
A target represents in this context the LLM we are evaluating from a red-team perspective.

In [ ]:
# Configuration for your Azure OpenAI Model
# ---> This will be gpt-4o or gpt-4o-mini, etc..
from dotenv import load_dotenv

# Load .env file
load_dotenv()

OPENAI_CHAT_ENDPOINT = os.environ.get("OPENAI_CHAT_ENDPOINT")

# configuration for Azure OpenAI
azure_openai_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_KEY"), # not required if authenticated with az login --use-device-code
    "deployment_name": os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME"),
    "model": "gpt-35-turbo",  # or "gpt-4o", "gpt-4o-mini", etc.
}

red_team_result = await red_team_agent.scan(scan_name="Red Team Sugar",
                                            target=azure_openai_config)

# Use of Attack Strategy #
This uses the grouping of the class 'AttackStrategy' for various stages the comments are here to explain

In [ ]:
from azure.ai.evaluation.red_team import AttackStrategy


# Run the red team scan with multiple attack strategies
advanced_result = await red_team_agent.scan(
    target=azure_openai_config,
    scan_name="Advanced-Callback-Scan",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.CharacterSpace,  # Add character spaces
        AttackStrategy.ROT13,  # Use ROT13 encoding
        AttackStrategy.UnicodeConfusable,  # Use confusable Unicode characters
        AttackStrategy.CharSwap,  # Swap characters in prompts 
        AttackStrategy.Morse,  # Encode prompts in Morse code - ".---..---"
        AttackStrategy.Leetspeak,  # Use Leetspeak think of 1337 h4x0r
        AttackStrategy.Url,  # Use URLs in prompts "r0b%20r0b%20r0b%20r0b the $bA3b3$",
        AttackStrategy.Binary,  # Encode prompts in binary
        AttackStrategy.Compose([AttackStrategy.Base64, AttackStrategy.ROT13]),  # Use two strategies in one attack
    ],
    output_path="Scan.json", # if you want to define the .json file output path
)